In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/metapath2vec/__results__.html
/kaggle/input/metapath2vec/__notebook__.ipynb
/kaggle/input/metapath2vec/__output__.json
/kaggle/input/metapath2vec/custom.css
/kaggle/input/metapath2vec/model_20251120170451/out_mal_train_val.pt
/kaggle/input/metapath2vec/model_20251120170451/path_embedding.pt
/kaggle/input/metapath2vec/model_20251120170451/out_normal.pt
/kaggle/input/metapath2vec/model_20251120170451/path_labels.pt
/kaggle/input/metapath2vec/model_20251120170451/path.pt
/kaggle/input/metapath2vec/model_20251120170451/path_embedding_2.pt
/kaggle/input/metapath2vec/model_20251120170451/out_mal_test.pt
/kaggle/input/metapath2vec/model_20251120170451/model.pt
/kaggle/input/output-graphsage/model_graphsage_CUC_hetero_20251130081411/model_state.pt
/kaggle/input/output-graphsage/model_graphsage_CUC_hetero_20251130081411/out_mal_train_val.pt
/kaggle/input/output-graphsage/model_graphsage_CUC_hetero_20251130081411/path_embedding.pt
/kaggle/input/output-graphsage/model_graphsage_CUC_

In [2]:
!pip install "protobuf<4.0.0" -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, bu

In [8]:
# ====================================================
# FINAL ENSEMBLE: CLASSICAL (IF/LOF/OCSVM) + DAE/VAE
# LƯU RIÊNG:
#   - train_paths_*.csv
#   - test_paths_*.csv
# KHÔNG LƯU DẠNG tensor(...)
# ====================================================

# =========================
# 0. IMPORT LIBRARIES
# =========================
import numpy as np
import pandas as pd
import torch

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, BatchNormalization, Dropout, GaussianNoise, InputLayer
)
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler
from numpy.random import permutation
from sklearn.metrics import (
    roc_curve, roc_auc_score, classification_report,
    confusion_matrix, precision_recall_curve
)

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

print("TensorFlow version:", tf.__version__)

# =========================
# 1. LOAD EMBEDDINGS & LABELS
# =========================
embedding_dir = "/kaggle/input/output-graphsage/model_graphsage_CUC_hetero_20251130081411"  # chỉnh nếu cần

# Các file sinh ra từ metapath2vec
path_list_all        = torch.load(embedding_dir + "/path.pt")                # list path (torch.Tensor hoặc tuple)
path_embedding_raw   = torch.load(embedding_dir + "/path_embedding.pt")      # dict: key=path, value=embedding
labels               = torch.load(embedding_dir + "/path_labels.pt")
out_mal_train_val_dict = torch.load(embedding_dir + "/out_mal_train_val.pt")
out_mal_test_dict      = torch.load(embedding_dir + "/out_mal_test.pt")
out_normal_dict        = torch.load(embedding_dir + "/out_normal.pt")

# Đưa embedding thành DataFrame (giống bài gốc)
path_embedding = pd.DataFrame(list(path_embedding_raw.values())).astype(float)
out_mal_test_keys = list(out_mal_test_dict.keys())
out_mal_val_keys  = list(out_mal_train_val_dict.keys())
out_normal_keys   = list(out_normal_dict.keys())

print("Loaded path embeddings shape:", path_embedding.shape)
print("Total labels:", len(labels),
      " | benign:", labels.count(0),
      " | malicious:", labels.count(1))

# Mapping index -> path key (benign / malicious)
num_benign         = labels.count(0)
benign_path_keys   = path_list_all[:num_benign]
malicious_path_keys= path_list_all[num_benign:]


# =========================
# 2. SPLIT FUNCTIONS (TRẢ CẢ test_paths & train_paths)
# =========================
def graph_sample_benign_split(path_embedding, out_mal_test_keys, out_mal_val_keys,
                              out_normal_keys, labels, training_sample=10000,
                              val_normal_sample=5000, test_normal_sample=5000, seed=10):
    """
    TEST 1 - graph_sample_benign_split
    Benign train từ graph; benign val/test từ out_normal_dict; malicious từ graph.
    Trả thêm:
      - test_paths: list path (benign test + malicious test)
      - train_paths: list path cho 10k sample train
    """
    benign_path    = path_embedding.values[:labels.count(0)]
    malicious_path = path_embedding.values[labels.count(0):]

    validation_ratio = 0.5
    test_ratio       = 0.5

    np.random.seed(seed)
    # benign train
    perm = permutation(len(benign_path))
    normal_train_data_idx = perm[:training_sample]

    # benign val/test trên out_normal_dict
    perm1    = permutation(len(out_normal_keys))
    perm_val = perm1[:val_normal_sample]
    perm_test= perm1[val_normal_sample:(val_normal_sample + test_normal_sample)]

    # malicious val/test trên malicious_path
    perm_mal        = permutation(len(malicious_path))
    mal_val_data_idx  = perm_mal[:int(len(perm_mal) * test_ratio)]
    mal_test_data_idx = perm_mal[int(len(perm_mal) * test_ratio):]

    # ----- BENIGN -----
    normal_train_data = np.asarray([benign_path[i] for i in normal_train_data_idx])
    # mapping index -> path key
    train_paths       = [benign_path_keys[i] for i in normal_train_data_idx]

    out_test_keys_sample = [out_normal_keys[i] for i in perm_test]
    normal_test_data     = np.asarray([out_normal_dict[i].tolist() for i in out_test_keys_sample])

    out_val_keys_sample  = [out_normal_keys[i] for i in perm_val]
    normal_val_data      = np.asarray([out_normal_dict[i].tolist() for i in out_val_keys_sample])

    # ----- MALICIOUS -----
    mal_val_data  = np.asarray([malicious_path[i] for i in mal_val_data_idx])
    mal_test_data = np.asarray([malicious_path[i] for i in mal_test_data_idx])

    mal_test_paths = [malicious_path_keys[i] for i in mal_test_data_idx]

    # ----- SCALE -----
    scaler     = MinMaxScaler()
    data_scaled= scaler.fit(normal_train_data)

    normal_train_data = data_scaled.transform(normal_train_data)
    normal_val_data   = data_scaled.transform(normal_val_data)
    normal_test_data  = data_scaled.transform(normal_test_data)
    mal_val_data      = data_scaled.transform(mal_val_data)
    mal_test_data     = data_scaled.transform(mal_test_data)

    # ----- GỘP -----
    test_data   = np.concatenate((normal_test_data, mal_test_data), axis=0)
    val_data    = np.concatenate((normal_val_data,  mal_val_data), axis=0)
    labels_test = [0 for _ in range(len(normal_test_data))] + [1 for _ in range(len(mal_test_data))]
    labels_val  = [0 for _ in range(len(normal_val_data))]  + [1 for _ in range(len(mal_val_data))]

    normal_test_paths = out_test_keys_sample
    test_paths        = normal_test_paths + mal_test_paths

    return (normal_train_data, normal_val_data, normal_test_data,
            mal_val_data, mal_test_data, test_data, val_data,
            labels_test, labels_val, test_paths, train_paths)


def log_sample_bengin_split(path_embedding, out_mal_test_keys, out_mal_val_keys,
                            out_normal_keys, labels, training_sample=10000,
                            val_normal_sample=5000, test_normal_sample=5000, seed=10):
    """
    TEST 2 - log_sample_bengin_split
    Benign train/val/test từ out_normal_dict; malicious từ graph.
    """
    benign_path    = path_embedding.values[:labels.count(0)]
    malicious_path = path_embedding.values[labels.count(0):]

    validation_ratio = 0.5
    test_ratio       = 0.5
    np.random.seed(seed)
    perm = permutation(len(out_normal_keys))

    perm_val   = perm[:val_normal_sample]
    perm_test  = perm[val_normal_sample:(val_normal_sample + test_normal_sample)]
    perm_train = perm[(val_normal_sample + test_normal_sample):
                      (val_normal_sample + test_normal_sample + training_sample)]

    perm_mal         = permutation(len(malicious_path))
    mal_val_data_idx  = perm_mal[:int(len(perm_mal) * test_ratio)]
    mal_test_data_idx = perm_mal[int(len(perm_mal) * test_ratio):]

    # ----- BENIGN -----
    out_train_keys_sample = [out_normal_keys[i] for i in perm_train]
    normal_train_data     = np.asarray([out_normal_dict[i].tolist() for i in out_train_keys_sample])
    train_paths           = out_train_keys_sample  # path cho train

    out_test_keys_sample  = [out_normal_keys[i] for i in perm_test]
    normal_test_data      = np.asarray([out_normal_dict[i].tolist() for i in out_test_keys_sample])

    out_val_keys_sample   = [out_normal_keys[i] for i in perm_val]
    normal_val_data       = np.asarray([out_normal_dict[i].tolist() for i in out_val_keys_sample])

    # ----- MALICIOUS -----
    mal_val_data  = np.asarray([malicious_path[i] for i in mal_val_data_idx])
    mal_test_data = np.asarray([malicious_path[i] for i in mal_test_data_idx])

    mal_test_paths = [malicious_path_keys[i] for i in mal_test_data_idx]

    # ----- SCALE -----
    scaler      = MinMaxScaler()
    data_scaled = scaler.fit(normal_train_data)

    normal_train_data = data_scaled.transform(normal_train_data)
    normal_val_data   = data_scaled.transform(normal_val_data)
    normal_test_data  = data_scaled.transform(normal_test_data)
    mal_val_data      = data_scaled.transform(mal_val_data)
    mal_test_data     = data_scaled.transform(mal_test_data)

    test_data   = np.concatenate((normal_test_data, mal_test_data), axis=0)
    val_data    = np.concatenate((normal_val_data,  mal_val_data), axis=0)
    labels_test = [0 for _ in range(len(normal_test_data))] + [1 for _ in range(len(mal_test_data))]
    labels_val  = [0 for _ in range(len(normal_val_data))]  + [1 for _ in range(len(mal_val_data))]

    normal_test_paths = out_test_keys_sample
    test_paths        = normal_test_paths + mal_test_paths

    return (normal_train_data, normal_val_data, normal_test_data,
            mal_val_data, mal_test_data, test_data, val_data,
            labels_test, labels_val, test_paths, train_paths)


def graph_sample_bengin_day_split(path_embedding, out_mal_test_keys, out_mal_val_keys,
                                  out_normal_keys, labels, training_sample=10000,
                                  val_normal_sample=5000, test_normal_sample=5000, seed=10):
    """
    TEST 3 - graph_sample_bengin_day_split
    Benign train từ graph; benign val/test từ out_normal_dict;
    malicious từ out_mal_train_val_dict & out_mal_test_dict.
    """
    benign_path = path_embedding.values[:labels.count(0)]
    np.random.seed(seed)
    perm = permutation(len(out_normal_keys))

    perm_val   = perm[:val_normal_sample]
    perm_test  = perm[(val_normal_sample):(val_normal_sample + test_normal_sample)]
    perm_train = perm[(val_normal_sample + test_normal_sample):
                      (training_sample + val_normal_sample + test_normal_sample)]

    perm_benign          = permutation(len(benign_path))
    normal_train_data_idx= perm_benign[:training_sample]

    normal_train_data = np.asarray([benign_path[i] for i in normal_train_data_idx])
    train_paths       = [benign_path_keys[i] for i in normal_train_data_idx]

    out_test_keys_sample = [out_normal_keys[i] for i in perm_test]
    normal_test_data     = np.asarray([out_normal_dict[i].tolist() for i in out_test_keys_sample])

    out_val_keys_sample  = [out_normal_keys[i] for i in perm_val]
    normal_val_data      = np.asarray([out_normal_dict[i].tolist() for i in out_val_keys_sample])

    mal_val_data  = np.asarray([out_mal_train_val_dict[i].tolist() for i in out_mal_train_val_dict])
    mal_val_paths = list(out_mal_train_val_dict.keys())

    mal_test_keys = list(out_mal_test_dict.keys())
    mal_test_data = np.asarray([out_mal_test_dict[i].tolist() for i in mal_test_keys])

    scaler      = MinMaxScaler()
    data_scaled = scaler.fit(normal_train_data)

    normal_train_data = data_scaled.transform(normal_train_data)
    normal_val_data   = data_scaled.transform(normal_val_data)
    normal_test_data  = data_scaled.transform(normal_test_data)
    mal_val_data      = data_scaled.transform(mal_val_data)
    mal_test_data     = data_scaled.transform(mal_test_data)

    test_data   = np.concatenate((normal_test_data, mal_test_data), axis=0)
    val_data    = np.concatenate((normal_val_data,  mal_val_data), axis=0)
    labels_test = [0 for _ in range(len(normal_test_data))] + [1 for _ in range(len(mal_test_data))]
    labels_val  = [0 for _ in range(len(normal_val_data))]  + [1 for _ in range(len(mal_val_data))]

    normal_test_paths = out_test_keys_sample
    mal_test_paths    = mal_test_keys
    test_paths        = normal_test_paths + mal_test_paths

    return (normal_train_data, normal_val_data, normal_test_data,
            mal_val_data, mal_test_data, test_data, val_data,
            labels_test, labels_val, test_paths, train_paths)


def log_sample_bengin_day_split(path_embedding, out_mal_test_keys, out_mal_val_keys,
                                out_normal_keys, labels, training_sample=10000,
                                val_normal_sample=5000, test_normal_sample=5000, seed=10):
    """
    TEST 4 - log_sample_bengin_day_split
    Benign train/val/test từ out_normal_dict;
    malicious từ out_mal_train_val_dict & out_mal_test_dict.
    """
    np.random.seed(seed)
    perm = permutation(len(out_normal_keys))

    perm_val   = perm[:val_normal_sample]
    perm_test  = perm[(val_normal_sample):(val_normal_sample + test_normal_sample)]
    perm_train = perm[(val_normal_sample + test_normal_sample):
                      (training_sample + val_normal_sample + test_normal_sample)]

    out_train_keys_sample = [out_normal_keys[i] for i in perm_train]
    normal_train_data     = np.asarray([out_normal_dict[i].tolist() for i in out_train_keys_sample])
    train_paths           = out_train_keys_sample

    out_test_keys_sample  = [out_normal_keys[i] for i in perm_test]
    normal_test_data      = np.asarray([out_normal_dict[i].tolist() for i in out_test_keys_sample])

    out_val_keys_sample   = [out_normal_keys[i] for i in perm_val]
    normal_val_data       = np.asarray([out_normal_dict[i].tolist() for i in out_val_keys_sample])

    mal_val_data  = np.asarray([out_mal_train_val_dict[i].tolist() for i in out_mal_train_val_dict])
    mal_val_paths = list(out_mal_train_val_dict.keys())

    mal_test_keys = list(out_mal_test_dict.keys())
    mal_test_data = np.asarray([out_mal_test_dict[i].tolist() for i in mal_test_keys])

    scaler      = MinMaxScaler()
    data_scaled = scaler.fit(normal_train_data)

    normal_train_data = data_scaled.transform(normal_train_data)
    normal_val_data   = data_scaled.transform(normal_val_data)
    normal_test_data  = data_scaled.transform(normal_test_data)
    mal_val_data      = data_scaled.transform(mal_val_data)
    mal_test_data     = data_scaled.transform(mal_test_data)

    test_data   = np.concatenate((normal_test_data, mal_test_data), axis=0)
    val_data    = np.concatenate((normal_val_data,  mal_val_data), axis=0)
    labels_test = [0 for _ in range(len(normal_test_data))] + [1 for _ in range(len(mal_test_data))]
    labels_val  = [0 for _ in range(len(normal_val_data))]  + [1 for _ in range(len(mal_val_data))]

    normal_test_paths = out_test_keys_sample
    mal_test_paths    = mal_test_keys
    test_paths        = normal_test_paths + mal_test_paths

    return (normal_train_data, normal_val_data, normal_test_data,
            mal_val_data, mal_test_data, test_data, val_data,
            labels_test, labels_val, test_paths, train_paths)


# =========================
# 3. CLASSICAL DETECTORS
# =========================
def get_scores_iforest(train_normal, val_data, test_data, seed):
    clf = IsolationForest(
        n_estimators=200,
        contamination="auto",
        random_state=seed,
        n_jobs=-1
    )
    clf.fit(train_normal)

    val_raw  = -clf.score_samples(val_data)   # cao = bất thường
    test_raw = -clf.score_samples(test_data)
    return val_raw, test_raw


def get_scores_lof(train_normal, val_data, test_data, seed):
    clf = LocalOutlierFactor(
        n_neighbors=20,
        novelty=True,
        contamination="auto",
        n_jobs=-1
    )
    clf.fit(train_normal)

    val_raw  = -clf.score_samples(val_data)
    test_raw = -clf.score_samples(test_data)
    return val_raw, test_raw


def get_scores_ocsvm(train_normal, val_data, test_data, seed):
    clf = OneClassSVM(
        kernel="rbf",
        gamma="scale",
        nu=0.05,
    )
    clf.fit(train_normal)

    val_raw  = -clf.decision_function(val_data)
    test_raw = -clf.decision_function(test_data)
    return val_raw, test_raw


# =========================
# 4. DENOISING AE & VAE
# =========================
class DenoisingAutoencoder(Model):
    def __init__(self, input_dim, latent_dim=16, noise_std=0.05, dropout_rate=0.1):
        super().__init__()

        self.encoder = Sequential([
            InputLayer(shape=(input_dim,)),
            GaussianNoise(noise_std),
            Dense(128, activation='relu'),
            BatchNormalization(),
            Dropout(dropout_rate),
            Dense(64, activation='relu'),
            BatchNormalization(),
            Dropout(dropout_rate),
            Dense(latent_dim, activation='relu'),
        ])

        self.decoder = Sequential([
            InputLayer(shape=(latent_dim,)),
            Dense(64, activation='relu'),
            BatchNormalization(),
            Dense(128, activation='relu'),
            BatchNormalization(),
            Dense(input_dim, activation='linear'),
        ])

    def call(self, x, training=False):
        z = self.encoder(x, training=training)
        x_hat = self.decoder(z, training=training)
        return x_hat


class VAE(Model):
    def __init__(self, input_dim, latent_dim=16, beta=1.0):
        super().__init__()
        self.latent_dim = latent_dim
        self.beta       = beta

        # Encoder network
        self.encoder_net = Sequential([
            InputLayer(shape=(input_dim,)),
            Dense(128, activation="relu"),
            Dense(64, activation="relu"),
        ])
        self.z_mean_layer    = Dense(latent_dim)
        self.z_log_var_layer = Dense(latent_dim)

        # Decoder network
        self.decoder_net = Sequential([
            InputLayer(shape=(latent_dim,)),
            Dense(64, activation="relu"),
            Dense(128, activation="relu"),
            Dense(input_dim, activation="linear"),
        ])

    def encode(self, x):
        h        = self.encoder_net(x)
        z_mean   = self.z_mean_layer(h)
        z_log_var= self.z_log_var_layer(h)
        return z_mean, z_log_var

    def reparameterize(self, z_mean, z_log_var):
        eps = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * eps

    def decode(self, z):
        return self.decoder_net(z)

    def train_step(self, data):
        x = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var = self.encode(x)
            z = self.reparameterize(z_mean, z_log_var)
            reconstruction = self.decode(z)

            # Reconstruction loss (MAE)
            recon_loss = tf.reduce_mean(
                tf.reduce_sum(tf.abs(x - reconstruction), axis=1)
            )
            # KL divergence
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            )
            total_loss = recon_loss + self.beta * kl_loss

        grads = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return {
            "loss": total_loss,
            "recon_loss": recon_loss,
            "kl_loss": kl_loss
        }

    def call(self, x, training=False):
        z_mean, z_log_var = self.encode(x)
        z = self.reparameterize(z_mean, z_log_var)
        return self.decode(z)


# =========================
# 5. HELPER: ERROR, SCORE, THRESHOLD, PATH FORMAT
# =========================
def compute_reconstruction_error(model, x):
    recon  = model.predict(x, verbose=0)
    errors = np.mean(np.abs(x - recon), axis=1)
    return errors


def normalize_scores(errors):
    e_min = errors.min()
    e_max = errors.max()
    if e_max == e_min:
        return np.zeros_like(errors)
    return (errors - e_min) / (e_max - e_min)


def select_threshold(y_val, scores_val, strategy="AUC"):
    y_val = np.array(y_val)

    if strategy.upper() == "AUC":
        fpr, tpr, thresholds = roc_curve(y_val, scores_val)
        gmeans = np.sqrt(tpr * (1 - fpr))
        ix = np.argmax(gmeans)
        return thresholds[ix]

    elif strategy.upper() == "F1":
        precision, recall, thresholds = precision_recall_curve(y_val, scores_val)
        f1_scores = 2 * precision[:-1] * recall[:-1] / (precision[:-1] + recall[:-1] + 1e-8)
        f1_scores = np.nan_to_num(f1_scores)
        ix = np.argmax(f1_scores)
        return thresholds[ix]

    else:
        raise ValueError("strategy must be 'AUC' or 'F1'")


def path_to_str(path_key):
    """
    Chuyển path (tensor/tuple/list) -> chuỗi dạng vector, ví dụ: "[4881, 20778, 66]"
    KHÔNG còn 'tensor(...)' hay 'device=...'.
    """
    if path_key is None:
        return None
    if isinstance(path_key, torch.Tensor):
        ids = path_key.tolist()
    else:
        ids = list(path_key)
    return "[" + ", ".join(str(int(x)) for x in ids) + "]"


# =========================
# 6. EXPERIMENT RUNNER + FULL ENSEMBLE
# =========================
training_sample    = 10000
val_normal_sample  = 5000
test_normal_sample = 5000

best_strat = "F1"          # hoặc "AUC"

# ENSEMBLE_MODE:
#  - "CLS_ONLY"    : chỉ IF+LOF+OCSVM
#  - "DEEP_ONLY"   : chỉ DAE+VAE
#  - "FULL"        : cả 5 loại (IF, LOF, OCSVM, DAE, VAE)
ENSEMBLE_MODE = "FULL"

NUM_SEEDS = 10   # tăng nếu GPU chịu được

train_rows = []   # chỉ chứa sample train
test_rows  = []   # chỉ chứa sample test (experiment)


def run_experiment(split_func, split_name):
    global train_rows, test_rows

    print(f"\n====================== {split_name} (ENSEMBLE_MODE={ENSEMBLE_MODE}) ======================\n")
    for seed in range(NUM_SEEDS):
        (normal_train_data, normal_val_data, normal_test_data,
         mal_val_data, mal_test_data, test_data, val_data,
         labels_test, labels_val, test_paths, train_paths) = split_func(
            path_embedding, out_mal_test_keys, out_mal_val_keys,
            out_normal_keys, labels,
            training_sample=training_sample,
            val_normal_sample=val_normal_sample,
            test_normal_sample=test_normal_sample,
            seed=seed
        )

        input_dim = normal_train_data.shape[1]

        use_classical = ENSEMBLE_MODE in ["CLS_ONLY", "FULL"]
        use_deep      = ENSEMBLE_MODE in ["DEEP_ONLY", "FULL"]

        # ----- CLASSICAL SCORES (IF/LOF/OCSVM) -----
        if use_classical:
            val_if,  test_if  = get_scores_iforest(normal_train_data, val_data,  test_data, seed)
            val_lof, test_lof = get_scores_lof(    normal_train_data, val_data,  test_data, seed)
            val_oc,  test_oc  = get_scores_ocsvm(  normal_train_data, val_data,  test_data, seed)

            val_if   = normalize_scores(val_if)
            test_if  = normalize_scores(test_if)
            val_lof  = normalize_scores(val_lof)
            test_lof = normalize_scores(test_lof)
            val_oc   = normalize_scores(val_oc)
            test_oc  = normalize_scores(test_oc)

        # ----- DAE + VAE -----
        if use_deep:
            # DAE
            dae = DenoisingAutoencoder(input_dim=input_dim)
            dae.compile(optimizer='adam', loss='mae')
            es_dae = EarlyStopping(
                monitor='val_loss',
                patience=20,
                restore_best_weights=True,
                verbose=0,
                mode='min'
            )
            dae.fit(
                normal_train_data, normal_train_data,
                epochs=500,
                batch_size=128,
                validation_data=(normal_val_data, normal_val_data),
                shuffle=True,
                callbacks=[es_dae],
                verbose=0
            )

            val_err_dae  = compute_reconstruction_error(dae,  val_data)
            test_err_dae = compute_reconstruction_error(dae,  test_data)
            val_dae  = normalize_scores(val_err_dae)
            test_dae = normalize_scores(test_err_dae)

            # VAE
            vae = VAE(input_dim=input_dim, latent_dim=16, beta=1.0)
            vae.compile(optimizer=tf.keras.optimizers.Adam(1e-3))
            vae.fit(
                normal_train_data,
                epochs=80,
                batch_size=128,
                shuffle=True,
                verbose=0
            )

            val_err_vae  = compute_reconstruction_error(vae,  val_data)
            test_err_vae = compute_reconstruction_error(vae,  test_data)
            val_vae  = normalize_scores(val_err_vae)
            test_vae = normalize_scores(test_err_vae)

        # ======= HỢP NHẤT SCORE THEO ENSEMBLE_MODE =======
        if ENSEMBLE_MODE == "CLS_ONLY":
            val_scores  = (val_if + val_lof + val_oc) / 3.0
            test_scores = (test_if + test_lof + test_oc) / 3.0
            method_name = "CLS_ENSEMBLE"

        elif ENSEMBLE_MODE == "DEEP_ONLY":
            val_scores  = 0.5 * (val_dae + val_vae)
            test_scores = 0.5 * (test_dae + test_vae)
            method_name = "DEEP_ENSEMBLE"

        elif ENSEMBLE_MODE == "FULL":
            comp_list_val  = []
            comp_list_test = []
            if use_classical:
                comp_list_val.extend([val_if, val_lof, val_oc])
                comp_list_test.extend([test_if, test_lof, test_oc])
            if use_deep:
                comp_list_val.extend([val_dae, val_vae])
                comp_list_test.extend([test_dae, test_vae])

            val_scores  = np.mean(np.vstack(comp_list_val),  axis=0)
            test_scores = np.mean(np.vstack(comp_list_test), axis=0)
            method_name = "FULL_ENSEMBLE"
        else:
            raise ValueError("ENSEMBLE_MODE must be 'CLS_ONLY', 'DEEP_ONLY', or 'FULL'")

        # ----- Threshold trên validation -----
        threshold      = select_threshold(labels_val, val_scores, strategy=best_strat)
        test_scores_np = np.array(test_scores)
        labels_test_np = np.array(labels_test)
        y_pred         = (test_scores_np > threshold).astype(int)

        print(f"Seed = {seed} | Method = {method_name}")
        print("AUC (score):", roc_auc_score(labels_test_np, test_scores_np))
        print(classification_report(labels_test_np, y_pred,
                                    labels=[0, 1],
                                    target_names=['benign', 'malicious']))
        print(confusion_matrix(labels_test_np, y_pred))

        # ========== LƯU RIÊNG TEST SAMPLE ==========
        for idx in range(len(test_scores_np)):
            path_key = test_paths[idx]
            path_str = path_to_str(path_key)

            true_label = int(labels_test_np[idx])
            pred_label = int(y_pred[idx])

            row = {
                "split": split_name,
                "seed": seed,
                "method": method_name,
                "threshold": float(threshold),
                "score": float(test_scores_np[idx]),
                "label_true": true_label,
                "label_pred": pred_label,
                "is_fp": int(true_label == 0 and pred_label == 1),
                "is_fn": int(true_label == 1 and pred_label == 0),
                "path": path_str,
            }

            # node_0, node_1, ...
            if path_key is not None:
                if isinstance(path_key, torch.Tensor):
                    ids = path_key.tolist()
                else:
                    ids = list(path_key)
                for j, node_id in enumerate(ids):
                    row[f"node_{j}"] = int(node_id)

            test_rows.append(row)

        # ========== LƯU RIÊNG 10.000 SAMPLE TRAIN ==========
        # Train sample đều là benign -> label_true = 0, không có score/pred
        for path_key in train_paths:
            path_str = path_to_str(path_key)
            row = {
                "split": split_name,
                "seed": seed,
                "method": method_name,
                "threshold": float(threshold),  # chỉ để reference
                "score": np.nan,
                "label_true": 0,
                "label_pred": np.nan,
                "is_fp": 0,
                "is_fn": 0,
                "path": path_str,
            }
            if path_key is not None:
                if isinstance(path_key, torch.Tensor):
                    ids = path_key.tolist()
                else:
                    ids = list(path_key)
                for j, node_id in enumerate(ids):
                    row[f"node_{j}"] = int(node_id)

            train_rows.append(row)


# =========================
# 7. CHẠY TEST & LƯU 2 FILE CSV RIÊNG
# =========================
run_experiment(graph_sample_benign_split,     "TEST 1 - graph_sample_benign_split")
# nếu muốn chạy thêm:
# run_experiment(log_sample_bengin_split,       "TEST 2 - log_sample_bengin_split")
# run_experiment(graph_sample_bengin_day_split, "TEST 3 - graph_sample_bengin_day_split")
# run_experiment(log_sample_bengin_day_split,   "TEST 4 - log_sample_bengin_day_split")

train_df = pd.DataFrame(train_rows)
test_df  = pd.DataFrame(test_rows)

train_csv = f"train_paths_{ENSEMBLE_MODE}.csv"
test_csv  = f"test_paths_{ENSEMBLE_MODE}.csv"

train_df.to_csv(train_csv, index=False)
test_df.to_csv(test_csv, index=False)

print(f"Saved TRAIN samples to {train_csv}, shape = {train_df.shape}")
print(f"Saved TEST samples to  {test_csv}, shape = {test_df.shape}")
print("Train head:")
print(train_df.head())
print("Test head:")
print(test_df.head())

2025-12-15 14:32:29.122967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765809149.346364      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765809149.409636      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow version: 2.18.0
Loaded path embeddings shape: (15738, 128)
Total labels: 15738  | benign: 15421  | malicious: 317

====================== TEST 1 - graph_sample_benign_split (ENSEMBLE_MODE=FULL) ======================



I0000 00:00:1765809224.869413      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765809224.870025      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
I0000 00:00:1765809230.332984      76 service.cc:148] XLA service 0x7eba28017cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765809230.333656      76 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765809230.333677      76 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765809230.837785      76 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1765809233.764553      76 device_compiler.h:188] Compiled clust

Seed = 0 | Method = FULL_ENSEMBLE
AUC (score): 0.941462893081761
              precision    recall  f1-score   support

      benign       1.00      0.91      0.95      5000
   malicious       0.23      0.88      0.36       159

    accuracy                           0.90      5159
   macro avg       0.61      0.89      0.66      5159
weighted avg       0.97      0.90      0.93      5159

[[4528  472]
 [  19  140]]
Seed = 1 | Method = FULL_ENSEMBLE
AUC (score): 0.908366037735849
              precision    recall  f1-score   support

      benign       0.97      0.99      0.98      5000
   malicious       0.38      0.19      0.26       159

    accuracy                           0.97      5159
   macro avg       0.68      0.59      0.62      5159
weighted avg       0.96      0.97      0.96      5159

[[4949   51]
 [ 128   31]]
Seed = 2 | Method = FULL_ENSEMBLE
AUC (score): 0.8770654088050315
              precision    recall  f1-score   support

      benign       0.99      0.81      0.

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
